In [2]:
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [3]:
#Read review data from csv
reviews = pd.read_csv("C:/Users/wgvan/Desktop/reviews.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
#drop irrelevant review columns
reviews = reviews.drop(columns=['id', 'workId', 'reviewId', 'postTime', 'episodesSeen', 'author', 'peopleFoundUseful'], axis=1)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin..."
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...


In [4]:
#Read genre data from csv
genres = pd.read_csv("C:/Users/wgvan/Desktop/anime.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
genres.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
#replace some punctuation and spaces with _ so that the names in the two files are the same
genres['name'].replace('\s+', '_',regex=True,inplace=True)
genres['name'].replace(':', '_',regex=True,inplace=True)
genres['name'].replace(';', '_',regex=True,inplace=True)
genres['name'].replace('&', '_',regex=True,inplace=True)
genres['name'].replace('\'', '',regex=True,inplace=True)
genres['name'].replace('!', '',regex=True,inplace=True)
genres.head()

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-92fda4bd457d>:2: DeprecationWarning: invalid escape sequence \s
  genres['name'].replace('\s+', '_',regex=True,inplace=True)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi_no_Na_wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal_Alchemist__Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins_Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama_#039_,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [12]:
#make a dictionary out of the two needed genre columns
genre_dict = pd.Series(genres.genre.values, index = genres.name).to_dict()
#add genre column to the reviews
reviews['genre'] = reviews['workName'].map(genre_dict)
reviews['genre'].replace(',', '',regex=True,inplace=True)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review,genre
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...,Action Adventure Comedy Drama Sci-Fi Space
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...,Action Drama Fantasy Sci-Fi
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin...",Comedy Drama Shounen Sports
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...,Adventure Comedy Drama Shounen Supernatural
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...,Comedy Fantasy Martial Arts Slice of Life


In [16]:
print(len(reviews.index))
reviews.isnull().sum(axis=0)
reviews.genre.str.split(expand=True).stack().value_counts()

135206


Comedy           43273
Action           41955
Drama            33250
Romance          31626
School           26446
Fantasy          25488
Supernatural     25048
Sci-Fi           22042
Shounen          21956
Adventure        19528
Life             16118
of               16118
Slice            16118
Mystery          13450
Psychological    11332
Seinen           11297
Power            10113
Super            10113
Ecchi             8740
Magic             8392
Horror            8096
Shoujo            8040
Harem             7330
Mecha             6751
Thriller          6447
Military          5508
Historical        5392
Game              4030
Music             3895
Demons            3578
Sports            3279
Martial           3082
Arts              3082
Parody            2764
Space             2723
Vampire           2720
Police            2607
Hentai            2307
Ai                1632
Dementia          1609
Samurai           1572
Josei             1349
Kids               930
Yaoi       

In [24]:
comedy = reviews[reviews['genre'].str.contains('Comedy', na = False)]
print(len(comedy.index))
comedy.head()

43273


,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review,genre
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...,Action Adventure Comedy Drama Sci-Fi Space
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin...",Comedy Drama Shounen Sports
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...,Adventure Comedy Drama Shounen Supernatural
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...,Comedy Fantasy Martial Arts Slice of Life
6,Kareshi_Kanojo_no_Jijou,8,8,6,7,9,8,Kare Kano was a romance anime that could have ...,Comedy Drama Romance School Shoujo Slice of Life


In [25]:
#remove punctiation and convert everything to lowercase
comedy['review_processed'] = comedy['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
comedy['review_processed'] = comedy['review_processed'].map(lambda x: x.lower())
comedy['review_processed'].head()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
<ipython-input-25-b183b0372ab2>:2: DeprecationWarning: invalid escape sequence \.
  comedy['review_processed'] = comedy['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
<ipython-input-25-b183b0372ab2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comedy['review_processed'] = comedy['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
<ipython-input-25-b183b0372ab2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

0    cowboy bebop is an episodic series by episodic...
2    first let me say that i\'m not a fan of boxing...
3    saiyuki is one of those animes that just grabs...
4    a comedy/romance based on the manga by rumiko ...
6    kare kano was a romance anime that could have ...
Name: review_processed, dtype: object

In [26]:
def toWords(sentences):
    for s in sentences:
        yield(gensim.utils.simple_preprocess(str(s), deacc=True))
        
data = comedy.review_processed.values.tolist()
data_words = list(toWords(data))

print(data_words[:1])

[['cowboy', 'bebop', 'is', 'an', 'episodic', 'series', 'by', 'episodic', 'mean', 'that', 'one', 'episode', 'doesn', 'necessarily', 'lead', 'or', 'follow', 'the', 'next', 'one', 'with', 'that', 'being', 'said', 'don', 'expect', 'lot', 'of', 'plot', 'in', 'this', 'series', 'you', 'watch', 'cowboy', 'bebop', 'for', 'the', 'characters', 'plain', 'and', 'simple', 'spike', 'faye', 'jet', 'ed', 'ein', 'are', 'what', 'make', 'this', 'anime', 'arguably', 'one', 'of', 'the', 'most', 'memorable', 'series', 'ever', 'produced', 'the', 'music', 'is', 'awesome', 'especially', 'if', 'you', 're', 'fan', 'of', 'jazz', 'the', 'animation', 'is', 'top', 'notch', 'considering', 'when', 'the', 'series', 'came', 'out', 'and', 'the', 'enjoyment', 'value', 'is', 'second', 'to', 'nonewhile', 'lot', 'of', 'people', 'want', 'that', 'similar', 'episode', 'feel', 'if', 'you', 're', 'the', 'kind', 'of', 'person', 'that', 'enjoys', 'fast', 'paced', 'anime', 'with', 'each', 'character', 'getting', 'his', 'or', 'her', '

In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [28]:
#remove stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model
nlp = en_core_web_sm.load()
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

C:\Users\wgvan\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.3). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[['mean', 'episode', 'necessarily', 'lead', 'follow', 'next', 'say', 'expect', 'lot', 'plot', 'character', 'plain', 'simple', 'spike', 'make', 'arguably', 'memorable', 'series', 'ever', 'produce', 'music', 'awesome', 'especially', 'fan', 'jazz', 'animation', 'consider', 'come', 'enjoyment', 'value', 'second', 'nonewhile', 'lot', 'people', 'want', 'similar', 'episode', 'feel', 'kind', 'person', 'enjoy', 'fast_pace', 'anime', 'character', 'get', 'spotlight', 'type', 'enjoy', 'fast_pace', 'fun', 'rather', 'deep', 'thought_provoking', 'dark', 'mysterious', 'watch', 'cowboy_bebop', 'vicious', 'character', 'able', 'bring', 'little', 'sinister', 'side', 'kind', 'show']]


In [30]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]]


In [31]:
# Build LDA model (alpha, eta default, num_topics = 10)
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

# Compute Coherence Score and perplexity
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Coherence Score:  0.3052326876198249

Perplexity:  -7.87874437371649


In [32]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.175397  0.003426       1        1  32.031349
8      0.150331 -0.005997       2        1  21.248329
9      0.144849  0.011919       3        1  12.881635
2      0.147558  0.006083       4        1  10.693447
5      0.093756  0.030917       5        1  10.113580
6      0.142602 -0.009383       6        1   9.015214
1      0.045986 -0.030561       7        1   2.095879
4     -0.085708 -0.034840       8        1   1.178570
0     -0.392747  0.223260       9        1   0.453720
3     -0.422025 -0.194823      10        1   0.288274, topic_info=           Term           Freq          Total Category  logprob  loglift
6     character  176248.000000  176248.000000  Default  30.0000  30.0000
2         anime  101310.000000  101310.000000  Default  29.0000  29.0000
926       movie   11562.000000   11562.000000  Default  28.0000  28.0000
46         show  116178.000000  116178.000000  Default  27.0000  27.0000
353        good   79508.000000   79508.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
2302     spirit     156.644257    1192.737305  Topic10  -4.9650   3.8190
1864       star     115.460785    1102.386719  Topic10  -5.2700   3.5927
688   adventure     117.745567    3389.817139  Topic10  -5.2504   2.4890
2783       fate      84.076454     848.007507  Topic10  -5.5872   3.5379
2833   familiar      85.178169    1539.345337  Topic10  -5.5742   2.9546

[777 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
188        1  0.204359     action
188        2  0.079521     action
188        3  0.076851     action
188        4  0.367298     action
188        5  0.242099     action
...      ...       ...        ...
1359       3  0.010566       youa
1359       5  0.019442       youa
1359       7  0.000423       youa
42076      6  0.996531  yuru_yuri
15073      6  0.996511     zombie

[2287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 10, 3, 6, 7, 2, 5, 1, 4])